In [ ]:
library(arules)
library(magrittr)
library(arulesViz)
library(dplyr)
library(chron)

In [ ]:
df <- read.csv ('../input/nba-games/games.csv', header = TRUE, dec ='.')
df2 <- read.csv ('../input/nba-games/ranking.csv', header = TRUE, dec ='.')

In [ ]:
df = df %>% mutate(GAME_DATE_EST = as.Date(GAME_DATE_EST, format = "%Y-%m-%d"),SEASON = as.factor(SEASON))
df2 <- unique(df2[ ,c('TEAM_ID','TEAM','CONFERENCE')])
df <- merge(df,df2,by.x = "HOME_TEAM_ID", by.y="TEAM_ID")
names(df)[names(df) == "TEAM"] <- "TEAM_HOME"
names(df)[names(df) == "CONFERENCE"] <- "CONFERENCE_HOME"
df <- merge(df,df2,by.x = "VISITOR_TEAM_ID", by.y="TEAM_ID")
names(df)[names(df) == "TEAM"] <- "TEAM_AWAY"
names(df)[names(df) == "CONFERENCE"] <- "CONFERENCE_AWAY"
df$homeWin <- df$HOME_TEAM_WINS == 1
df$awayWin <- df$HOME_TEAM_WINS == 0
df$conferenceHomeEast <- df$CONFERENCE_HOME == 'East'
df$conferenceHomeWest <- df$CONFERENCE_HOME == 'West'
df$conferenceAwayEast <- df$CONFERENCE_AWAY == 'East'
df$conferenceAwayWest <- df$CONFERENCE_AWAY == 'West'
df$moreFG_PCT_home = df$FG_PCT_home > df$FG_PCT_away
df$moreFG_PCT_away = df$FG_PCT_home < df$FG_PCT_away
df$drawFG_PCT = df$FG_PCT_home == df$FG_PCT_away
df$moreFT_PCT_home = df$FT_PCT_home > df$FT_PCT_away
df$moreFT_PCT_away = df$FT_PCT_home < df$FT_PCT_away
df$drawFT_PCT = df$FT_PCT_home == df$FT_PCT_away
df$moreFG3_PCT_home = df$FG3_PCT_home > df$FG3_PCT_away
df$moreFG3_PCT_away = df$FG3_PCT_home < df$FG3_PCT_away
df$drawFG3_PCT = df$FG3_PCT_home == df$FG3_PCT_away
df$moreAST_home = df$AST_home > df$AST_away
df$moreAST_away = df$AST_home < df$AST_away
df$drawAST = df$AST_home == df$AST_away
df$moreREB_home = df$REB_home > df$REB_away
df$moreREB_away = df$REB_home < df$REB_away
df$drawREB = df$REB_home == df$REB_away
df$weekday <- weekdays(df$GAME_DATE_EST)
df$playoff = ifelse(df$SEASON == '2003',df$GAME_DATE_EST > '2004-04-16',
             ifelse(df$SEASON == '2004',df$GAME_DATE_EST > '2005-04-22',
             ifelse(df$SEASON == '2005',df$GAME_DATE_EST > '2006-04-21',
             ifelse(df$SEASON == '2006',df$GAME_DATE_EST > '2007-04-20',
             ifelse(df$SEASON == '2007',df$GAME_DATE_EST > '2008-04-18',
             ifelse(df$SEASON == '2008',df$GAME_DATE_EST > '2009-04-17',
             ifelse(df$SEASON == '2009',df$GAME_DATE_EST > '2010-04-16',
             ifelse(df$SEASON == '2010',df$GAME_DATE_EST > '2011-04-15',
             ifelse(df$SEASON == '2011',df$GAME_DATE_EST > '2012-04-27',
             ifelse(df$SEASON == '2012',df$GAME_DATE_EST > '2013-04-19',
             ifelse(df$SEASON == '2013',df$GAME_DATE_EST > '2014-04-18',
             ifelse(df$SEASON == '2014',df$GAME_DATE_EST > '2015-04-17',
             ifelse(df$SEASON == '2015',df$GAME_DATE_EST > '2016-04-15',
             ifelse(df$SEASON == '2016',df$GAME_DATE_EST > '2017-04-14',
             ifelse(df$SEASON == '2017',df$GAME_DATE_EST > '2018-04-13',
             ifelse(df$SEASON == '2018',df$GAME_DATE_EST > '2019-04-12',
             FALSE))))))))))))))))
df$regularSeason = !df$playoff
df = df %>% mutate(weekday = as.factor(weekday))
df <- df[ , !(names(df) %in% c('TEAM_ID_home','TEAM_ID_away','GAME_STATUS_TEXT','VISITOR_TEAM_ID','HOME_TEAM_ID','HOME_TEAM_WINS','GAME_ID','CONFERENCE_HOME','CONFERENCE_AWAY'))]
#quito los registros con valores desconocidos
df <- df[rowSums(is.na(df)) == 0,]
df <- discretizeDF(df,default = list(method = "interval", breaks = 10))
df

Funciones útiles

Quitar las reglas redundantes

In [ ]:
sacarRedundantes <- function (reglas) {
    subset.matrix <- is.subset(reglas,reglas)
    subset.matrix[lower.tri(subset.matrix, diag=T)] <- F
    redundant <- apply(subset.matrix, 2, any)
    reglas.pruned <- reglas[!redundant]
    return (reglas.pruned)
}

In [ ]:
obtenerReglas <- function(df,minsupport=0.05,confianza=0.9){
    reglas <- apriori(df, parameter=list(support=minsupport,confidence = confianza))
    reglas <- sacarRedundantes(reglas)
    inspect(sort(reglas,by=c('lift','confidence')))
    return (reglas)
}

Procedimiento para graficar 

In [ ]:
graficarReglas <- function(reglas){
    plot(reglas, jitter = 0)
    plot(reglas, method="grouped")
    plot(reglas, method="graph")
}

In [ ]:
guardarGraficosReglas <- function(reglas){
    name <- deparse(substitute(reglas))
    file_names <- sprintf('data%02d_%s.jpeg', 1:3, name)

    jpeg(file_names[[1]])
    plot(reglas, jitter = 0)
    dev.off()
    
    jpeg(file_names[[2]])
    plot(reglas, method="grouped")
    dev.off()
    
    jpeg(file_names[[3]])
    plot(reglas, method="graph")
    dev.off()
}

General

In [ ]:
reglas <- obtenerReglas(df)

Hacemos gráficos

In [ ]:
graficarReglas(reglas)

Análisis temporada regular

In [ ]:
dfRegular = filter(df,regularSeason)
dfRegular <- dfRegular[, !(names(dfRegular) %in% c('playoff','regularSeason'))]

In [ ]:
obtenerReglas(dfRegular,0.04,0.6)

Análisis playoff

In [ ]:
dfPlayoff = filter(df,playoff)
dfPlayoff <- dfPlayoff[, !(names(dfPlayoff) %in% c('playoff','regularSeason'))]

In [ ]:
obtenerReglas(dfPlayoff,0.045,0.6)

Análisis temporada 2017-2018

In [ ]:
df2017 = filter(df,SEASON == '2017')
df2017 <- df2017[, !(names(df2017) %in% c('SEASON'))]

In [ ]:
reglas2017 <- obtenerReglas(df2017,0.07,0.6)
reglas2017

In [ ]:
dfWest2017 = filter(df,conferenceHomeWest & conferenceAwayWest & SEASON == '2017')
dfWest2017 <- dfWest2017[ , !(names(dfWest2017) %in% c('conferenceHomeWest','conferenceAwayWest','SEASON'))]

In [ ]:
reglasW2017 <- obtenerReglas(dfWest2017,0.06,0.6)
reglasW2017

In [ ]:
dfEast2017 = filter(df,conferenceHomeEast & conferenceAwayEast & SEASON == '2017')
dfEast2017 <- dfEast2017[ , !(names(dfEast2017) %in% c('conferenceHomeEast','conferenceAwayEast','SEASON'))]

In [ ]:
reglasE2017 <- obtenerReglas(dfEast2017,0.05,0.6)
reglasE2017

Partidos entre las conferencias

In [ ]:
dfBothConferences2017 = filter(df,((conferenceHomeEast & conferenceAwayWest) | (conferenceHomeWest & conferenceAwayEast)) & SEASON == '2017')
dfBothConferences2017 <- dfBothConferences2017[ , !(names(dfBothConferences2017) %in% c('SEASON'))]

In [ ]:
reglasB2017 <- obtenerReglas(dfBothConferences2017,0.065,0.6)
reglasB2017

Repetimos el análisis en la temporada 2018-2019

In [ ]:
df2018 = filter(df,SEASON == '2018')
df2018 <- df2018[, !(names(df2018) %in% c('SEASON'))]

In [ ]:
obtenerReglas(df2018,0.06,0.6)

Reglas para la conferencia oeste

In [ ]:
dfWest2018 = filter(df,conferenceHomeWest & conferenceAwayWest & SEASON == '2018')
dfWest2018 <- dfWest2018[ , !(names(dfWest2018) %in% c('conferenceHomeWest','conferenceAwayWest','SEASON'))]

In [ ]:
obtenerReglas(dfWest2018,0.06,0.6)

Reglas para la conferencia este

In [ ]:
dfEast2018 = filter(df,conferenceHomeEast & conferenceAwayEast & SEASON == '2018')
dfEast2018 <- dfEast2018[ , !(names(dfEast2018) %in% c('conferenceHomeEast','conferenceAwayEast','SEASON'))]

In [ ]:
obtenerReglas(dfEast2018,0.06,0.6)

Reglas para los partidos entre conferencias

In [ ]:
dfBothConferences2018 = filter(df,((conferenceHomeEast & conferenceAwayWest) | (conferenceHomeWest & conferenceAwayEast)) & SEASON == '2018')
dfBothConferences2018 <- dfBothConferences2018[ , !(names(dfBothConferences2018) %in% c('SEASON'))]

In [ ]:
obtenerReglas(dfBothConferences2018,0.07,0.6)

## Calculamos el soporte, confianza y lift** de las reglas del 2017-2018 para las reglas del 2018-2019

In [ ]:
calcularSoporte <- function(data,regla){
    local <- sum(regla)/nrow(data)
    return (local)
}
calcularConfianza <- function(data,regla,reglaLHS){
    local <- (sum(regla))/(sum(reglaLHS))
    return (local)
}
calcularLift <- function(data,regla,reglaLHS,reglaRHS){
    local <- calcularSoporte(data,regla)/(calcularSoporte(data,reglaLHS)*calcularSoporte(data,reglaRHS))
    return (local)
}

### Reglas correspondientes a la temporada 2017-2018 evaluadas en el 2018-2019

In [ ]:
data = df2017

#  Reglas correspondientes a la temporada 2017-2018 
reglas = list(data$moreFG_PCT_away & data$FG_PCT_away=="[0.502,0.545)"
, data$FG3_PCT_home=="[0.2,0.3)" & data$moreFG3_PCT_away
, data$FG3_PCT_home=="[0.4,0.5)" & data$moreFG3_PCT_home
, data$AST_away=="[25,29.2)" & data$moreAST_away
, data$FG3_PCT_away=="[0.4,0.5)" & data$moreFG3_PCT_away
, data$moreAST_away & data$awayWin
, data$AST_away=="[25,29.2)" & data$moreFT_PCT_away & data$awayWin
, data$PTS_home=="[115,128)" & data$homeWin
, data$moreFG3_PCT_home & data$homeWin
, data$moreAST_home & data$homeWin)
print("Soporte Reglas correspondientes a la temporada 2017-2018")
for(i in reglas){print(calcularSoporte(data, i))}

reglasLHS = data$FG_PCT_away=="[0.502,0.545)"
reglasLHS2 = data$FG3_PCT_home=="[0.2,0.3)"
reglasLHS3 = data$FG3_PCT_home=="[0.4,0.5)"
reglasLHS4 = data$AST_away=="[25,29.2)"
reglasLHS5 = data$FG3_PCT_away=="[0.4,0.5)"
reglasLHS6 = data$moreAST_away
reglasLHS7 = data$AST_away=="[25,29.2)"
reglasLHS8 = data$PTS_home=="[115,128)"
reglasLHS9 = data$moreFG3_PCT_home
reglasLHS10 = data$moreAST_home

reglasRHS = data$moreFG_PCT_away 
reglasRHS2 =  data$moreFG3_PCT_away
reglasRHS3 =  data$moreFG3_PCT_home
reglasRHS4 =  data$moreAST_away
reglasRHS5 =  data$moreFG3_PCT_away
reglasRHS6 =  data$awayWin
reglasRHS7 =  data$moreFT_PCT_away & data$awayWin
reglasRHS8 =  data$homeWin
reglasRHS9 =  data$homeWin
reglasRHS10 =  data$homeWin

print("Confianza Reglas correspondientes a la temporada 2017-2018")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))
print(calcularConfianza(data,reglas[[ 5 ]],reglasLHS5))
print(calcularConfianza(data,reglas[[ 6 ]],reglasLHS6))
print(calcularConfianza(data,reglas[[ 7 ]],reglasLHS7))
print(calcularConfianza(data,reglas[[ 8 ]],reglasLHS8))
print(calcularConfianza(data,reglas[[ 9 ]],reglasLHS9))
print(calcularConfianza(data,reglas[[ 10 ]],reglasLHS10))

print("Lift Reglas correspondientes a la temporada 2017-2018")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))
print(calcularLift(data,reglas[[ 5 ]],reglasLHS5,reglasRHS5))
print(calcularLift(data,reglas[[ 6 ]],reglasLHS6,reglasRHS6))
print(calcularLift(data,reglas[[ 7 ]],reglasLHS7,reglasRHS7))
print(calcularLift(data,reglas[[ 8 ]],reglasLHS8,reglasRHS8))
print(calcularLift(data,reglas[[ 9 ]],reglasLHS9,reglasRHS9))
print(calcularLift(data,reglas[[ 10 ]],reglasLHS10,reglasRHS10))

In [ ]:
data = df2018

reglas = list(data$moreFG_PCT_away & data$FG_PCT_away=="[0.502,0.545)"
, data$FG3_PCT_home=="[0.2,0.3)" & data$moreFG3_PCT_away
, data$FG3_PCT_home=="[0.4,0.5)" & data$moreFG3_PCT_home
, data$AST_away=="[25,29.2)" & data$moreAST_away
, data$FG3_PCT_away=="[0.4,0.5)" & data$moreFG3_PCT_away
, data$moreAST_away & data$awayWin
, data$AST_away=="[25,29.2)" & data$moreFT_PCT_away & data$awayWin
, data$PTS_home=="[115,128)" & data$homeWin
, data$moreFG3_PCT_home & data$homeWin
, data$moreAST_home & data$homeWin)

print("Soporte Reglas correspondientes a la temporada 2018-2019")
for(i in reglas){print(calcularSoporte(data, i))}


reglasLHS = data$FG_PCT_away=="[0.502,0.545)"
reglasLHS2 = data$FG3_PCT_home=="[0.2,0.3)"
reglasLHS3 = data$FG3_PCT_home=="[0.4,0.5)"
reglasLHS4 = data$AST_away=="[25,29.2)"
reglasLHS5 = data$FG3_PCT_away=="[0.4,0.5)"
reglasLHS6 = data$moreAST_away
reglasLHS7 = data$AST_away=="[25,29.2)"
reglasLHS8 = data$PTS_home=="[115,128)"
reglasLHS9 = data$moreFG3_PCT_home
reglasLHS10 = data$moreAST_home

reglasRHS = data$moreFG_PCT_away 
reglasRHS2 =  data$moreFG3_PCT_away
reglasRHS3 =  data$moreFG3_PCT_home
reglasRHS4 =  data$moreAST_away
reglasRHS5 =  data$moreFG3_PCT_away
reglasRHS6 =  data$awayWin
reglasRHS7 =  data$moreFT_PCT_away & data$awayWin
reglasRHS8 =  data$homeWin
reglasRHS9 =  data$homeWin
reglasRHS10 =  data$homeWin

print("Confianza Reglas correspondientes a la temporada 2018-2019")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))
print(calcularConfianza(data,reglas[[ 5 ]],reglasLHS5))
print(calcularConfianza(data,reglas[[ 6 ]],reglasLHS6))
print(calcularConfianza(data,reglas[[ 7 ]],reglasLHS7))
print(calcularConfianza(data,reglas[[ 8 ]],reglasLHS8))
print(calcularConfianza(data,reglas[[ 9 ]],reglasLHS9))
print(calcularConfianza(data,reglas[[ 10 ]],reglasLHS10))

print("Lift Reglas correspondientes a la temporada 2018-2019")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))
print(calcularLift(data,reglas[[ 5 ]],reglasLHS5,reglasRHS5))
print(calcularLift(data,reglas[[ 6 ]],reglasLHS6,reglasRHS6))
print(calcularLift(data,reglas[[ 7 ]],reglasLHS7,reglasRHS7))
print(calcularLift(data,reglas[[ 8 ]],reglasLHS8,reglasRHS8))
print(calcularLift(data,reglas[[ 9 ]],reglasLHS9,reglasRHS9))
print(calcularLift(data,reglas[[ 10 ]],reglasLHS10,reglasRHS10))

### Reglas correspondientes a partidos  en  la  conferencia oeste 2017-2018 evaluadas en el 2018-2019

In [ ]:
data = dfWest2017

reglas = list(data$PTS_away=="[114,128)" & data$awayWin
, data$moreAST_away & data$awayWin
, data$FG3_PCT_away=="[0.2,0.3)" & data$homeWin
, data$moreREB_home & data$homeWin)

print("Soporte Reglas correspondientes a la conferencia oeste 2017-2018")
for(i in reglas){print(calcularSoporte(data, i))}


reglasLHS = data$PTS_away=="[114,128)"
reglasLHS2 = data$moreAST_away
reglasLHS3 = data$FG3_PCT_away=="[0.2,0.3)" 
reglasLHS4 = data$moreREB_home     

reglasRHS = data$awayWin
reglasRHS2 =  data$awayWin
reglasRHS3 =  data$homeWin
reglasRHS4 =  data$homeWin


print("Confianza Reglas correspondientes a la conferencia oeste 2017-2018")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))


print("Lift Reglas correspondientes a la conferencia oeste 2017-2018")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))


In [ ]:
data = dfWest2018

reglas = list(data$PTS_away=="[114,128)"        & data$awayWin
, data$moreAST_away              & data$awayWin
, data$FG3_PCT_away=="[0.2,0.3)"    & data$homeWin
, data$moreREB_home              & data$homeWin)

print("Soporte Reglas correspondientes a la conferencia oeste 2018-2019")
for(i in reglas){print(calcularSoporte(data, i))}


reglasLHS = data$PTS_away=="[114,128)"
reglasLHS2 = data$moreAST_away
reglasLHS3 = data$FG3_PCT_away=="[0.2,0.3)" 
reglasLHS4 = data$moreREB_home     

reglasRHS = data$awayWin
reglasRHS2 =  data$awayWin
reglasRHS3 =  data$homeWin
reglasRHS4 =  data$homeWin


print("Confianza Reglas correspondientes a la conferencia oeste 2018-2019")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))


print("Lift Reglas correspondientes a la conferencia oeste 2018-2019")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))


### Reglas correspondientes a partidos  en  la  conferencia Este 2017-2018 evaluadas en el 2018-2019

In [ ]:
data = dfEast2017

reglas = list(data$FG_PCT_away=="[0.459,0.502)" & data$REB_away=="[43.8,50)" & data$awayWin
, data$REB_away=="[50,56.2)" & data$awayWin 
, data$PTS_home=="[115,128)" & data$homeWin  
, data$moreFG3_PCT_home & data$homeWin)

print("Soporte Reglas correspondientes a la conferencia este 2017-2018")
for(i in reglas){print(calcularSoporte(data, i))}


reglasLHS = data$FG_PCT_away=="[0.459,0.502)" & data$REB_away=="[43.8,50)" 
reglasLHS2 = data$REB_away=="[50,56.2)"
reglasLHS3 = data$PTS_home=="[115,128)" 
reglasLHS4 = data$moreFG3_PCT_home

reglasRHS = data$awayWin
reglasRHS2 =  data$awayWin 
reglasRHS3 =  data$homeWin  
reglasRHS4 =  data$homeWin  

print("Confianza Reglas correspondientes a la conferencia este 2017-2018")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))

print("Lift Reglas correspondientes a la conferencia este 2017-2018")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))

In [ ]:
data = dfEast2018

reglas = list(data$FG_PCT_away=="[0.459,0.502)" & data$REB_away=="[43.8,50)" & data$awayWin
, data$REB_away=="[50,56.2)" & data$awayWin 
, data$PTS_home=="[115,128)" & data$homeWin  
, data$moreFG3_PCT_home & data$homeWin)

print("Soporte Reglas correspondientes a la conferencia este 2018-2019")
for(i in reglas){print(calcularSoporte(data, i))}


reglasLHS = data$FG_PCT_away=="[0.459,0.502)" & data$REB_away=="[43.8,50)" 
reglasLHS2 = data$REB_away=="[50,56.2)"
reglasLHS3 = data$PTS_home=="[115,128)" 
reglasLHS4 = data$moreFG3_PCT_home

reglasRHS = data$awayWin
reglasRHS2 =  data$awayWin 
reglasRHS3 =  data$homeWin  
reglasRHS4 =  data$homeWin  

print("Confianza Reglas correspondientes a la conferencia este 2018-2019")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))

print("Lift Reglas correspondientes a la conferencia este 2018-2019")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))

### Reglas correspondientes a partidos de ambas conferencias en 2017-2018 evaluadas en el 2018-2019

In [ ]:
data = dfBothConferences2017

reglas = list(data$PTS_away=="[114,128)"        & data$awayWin  
, data$FG_PCT_away=="[0.459,0.502)" & data$moreREB_away              & data$awayWin   
, data$moreFG_PCT_home & data$homeWin             
, data$PTS_away=="[87,100)"  & data$homeWin)

print("Soporte Reglas correspondientes a ambas conferencias en 2017-2018")
for(i in reglas){print(calcularSoporte(data, i))}

reglasLHS = data$PTS_away=="[114,128)"
reglasLHS2 = data$FG_PCT_away=="[0.459,0.502)" & data$moreREB_away 
reglasLHS3 = data$moreFG_PCT_home
reglasLHS4 = data$PTS_away=="[87,100)" 

reglasRHS = data$awayWin  
reglasRHS2 =  data$awayWin 
reglasRHS3 =  data$homeWin  
reglasRHS4 =  data$homeWin   

print("Confianza Reglas correspondientes a ambas conferencias en 2017-2018")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))


print("Lift Reglas correspondientes a ambas conferencias en 2017-2018")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))

In [ ]:
data = dfBothConferences2018

reglas = list(data$PTS_away=="[114,128)"        & data$awayWin  
, data$FG_PCT_away=="[0.459,0.502)" & data$moreREB_away              & data$awayWin   
, data$moreFG_PCT_home & data$homeWin             
, data$PTS_away=="[87,100)"  & data$homeWin)

print("Soporte Reglas correspondientes a ambas conferencias en 2018-2019")
for(i in reglas){print(calcularSoporte(data, i))}

reglasLHS = data$PTS_away=="[114,128)"
reglasLHS2 = data$FG_PCT_away=="[0.459,0.502)" & data$moreREB_away 
reglasLHS3 = data$moreFG_PCT_home
reglasLHS4 = data$PTS_away=="[87,100)" 

reglasRHS = data$awayWin  
reglasRHS2 =  data$awayWin 
reglasRHS3 =  data$homeWin  
reglasRHS4 =  data$homeWin   

print("Confianza Reglas correspondientes a ambas conferencias en 2018-2019")
print(calcularConfianza(data,reglas[[ 1 ]],reglasLHS))
print(calcularConfianza(data,reglas[[ 2 ]],reglasLHS2))
print(calcularConfianza(data,reglas[[ 3 ]],reglasLHS3))
print(calcularConfianza(data,reglas[[ 4 ]],reglasLHS4))


print("Lift Reglas correspondientes a ambas conferencias en 2018-2019")
print(calcularLift(data,reglas[[ 1 ]],reglasLHS,reglasRHS))
print(calcularLift(data,reglas[[ 2 ]],reglasLHS2,reglasRHS2))
print(calcularLift(data,reglas[[ 3 ]],reglasLHS3,reglasRHS3))
print(calcularLift(data,reglas[[ 4 ]],reglasLHS4,reglasRHS4))

### Graficos reglas significativas en la temporada 2017-2018

In [ ]:
rSignificativas2017 <- c(subset(reglas2017, subset = ((rhs %in% "moreFG_PCT_away")&(lhs %in% "FG_PCT_away=[0.502,0.545)")))
           , subset(reglas2017, subset = ((rhs %in% "moreFG3_PCT_away" & (lhs %in% "FG3_PCT_home=[0.2,0.3)"))))
           , subset(reglas2017, subset = ((rhs %in% "moreFG3_PCT_home")&(lhs %in% "FG3_PCT_home=[0.4,0.5)")))
           , subset(reglas2017, subset = ((rhs %in% "moreAST_away")&(lhs %in% "AST_away=[25,29.2)")))
           , subset(reglas2017, subset = ((rhs %in% "moreFG3_PCT_away")&(lhs %in% "FG3_PCT_away=[0.4,0.5)")))
           , subset(reglas2017, subset = ((rhs %in% "awayWin")&(lhs %in% "moreAST_away")))
           , subset(reglas2017, subset = ((rhs %in% "awayWin")&(lhs %ain% c("AST_away=[25,29.2)","moreFT_PCT_away"))))
           , subset(reglas2017, subset = ((rhs %in% "homeWin")&(lhs %in% "PTS_home=[115,128)")))
           , subset(reglas2017, subset = ((rhs %in% "homeWin")&(lhs %in% "moreFG3_PCT_home")))
           , subset(reglas2017, subset = ((rhs %in% "homeWin")&(lhs %in% "moreAST_home")))     
          )
graficarReglas(rSignificativas2017)

In [ ]:
rSignificativas2017W <- c(subset(reglasW2017, subset = ((rhs %in% "awayWin")&(lhs %in% "PTS_away=[114,128)")))
           , subset(reglasW2017, subset = ((rhs %in% "awayWin" & (lhs %in% "moreAST_away"))))
           , subset(reglasW2017, subset = ((rhs %in% "homeWin")&(lhs %in% "FG3_PCT_away=[0.2,0.3)")))
           , subset(reglasW2017, subset = ((rhs %in% "homeWin")&(lhs %in% "moreREB_home")))     
          )
graficarReglas(rSignificativas2017W)

In [ ]:
rSignificativas2017E <- c(subset(reglasE2017, subset = ((rhs %in% "awayWin")&(lhs %ain% c("FG_PCT_away=[0.459,0.502)","REB_away=[43.8,50)"))))
           , subset(reglasE2017, subset = ((rhs %in% "awayWin" & (lhs %in% "REB_away=[50,56.2)"))))
           , subset(reglasE2017, subset = ((rhs %in% "homeWin")&(lhs %in% "PTS_home=[115,128)")))
           , subset(reglasE2017, subset = ((rhs %in% "homeWin")&(lhs %in% "moreFG3_PCT_home")))     
          )
graficarReglas(rSignificativas2017E)

In [ ]:
rSignificativas2017B <- c(subset(reglasB2017, subset = ((rhs %in% "awayWin")&(lhs %ain% c("FG_PCT_away=[0.459,0.502)","moreREB_away"))))
           , subset(reglasB2017, subset = ((rhs %in% "awayWin" & (lhs %in% "PTS_away=[114,128)"))))
           , subset(reglasB2017, subset = ((rhs %in% "homeWin")&(lhs %in% "moreFG_PCT_home")))
           , subset(reglasB2017, subset = ((rhs %in% "homeWin")&(lhs %in% "PTS_away=[87,100) ")))     
          )
graficarReglas(rSignificativas2017B)